<a href="https://colab.research.google.com/github/FireShadow05/DevRev_InterIIT_PS/blob/master/Retriever/BM25_with_LEDR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rank_bm25
!pip install beir
!pip install tensorflow_text

In [7]:
import numpy as np
import logging
import pathlib, os
import torch
import pickle
import gc
import sys

from beir import util, LoggingHandler
from beir.retrieval import models
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES

In [13]:
def top_k_accuracy(
        qrels, 
        results, 
        k_values) :
    
    top_k_acc = {}
    
    for k in k_values:
        top_k_acc[f"Accuracy@{k}"] = 0.0
    
    k_max, top_hits = max(k_values), {}
    
    for query_id, doc_scores in results.items():
        top_hits[query_id] = [item[0] for item in sorted(doc_scores.items(), key=lambda item: item[1], reverse=True)[0:k_max]]
    
    for query_id in top_hits:
        if query_id not in qrels.keys():
            continue
        query_relevant_docs = set([doc_id for doc_id in qrels[query_id] if qrels[query_id][doc_id] > 0])
        for k in k_values:
            for relevant_doc_id in query_relevant_docs:
                if relevant_doc_id in top_hits[query_id][0:k]:
                    
                    top_k_acc[f"Accuracy@{k}"] += 1.0
                    break

    for k in k_values:
        top_k_acc[f"Accuracy@{k}"] = round(top_k_acc[f"Accuracy@{k}"]/len(qrels), 5)

    return top_k_acc

In [8]:
from urllib.request import urlopen
import json
url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json"
response = urlopen(url)
data_json = json.loads(response.read())['data']
corpus = {}
queries = {}
qrels = {}
paras = []
id = 0
for i in range(len(data_json)):
    for j in range(len(data_json[i]['paragraphs'])):
          for k in range(len(data_json[i]['paragraphs'][j]['qas'])):
              queries[data_json[i]['paragraphs'][j]['qas'][k]['id']] = data_json[i]['paragraphs'][j]['qas'][k]['question']
              if not data_json[i]['paragraphs'][j]['qas'][k]['is_impossible']:
                  qrels[data_json[i]['paragraphs'][j]['qas'][k]['id']] = {str(id) : 1} 
          
          corpus[str(id)] = {
              'title' : data_json[i]['title'],
           'text' : data_json[i]['paragraphs'][j]['context']
          }
          paras.append(data_json[i]['paragraphs'][j]['context'])
          id += 1

In [9]:
from rank_bm25 import BM25Okapi
tokenized_corpus = [doc.split(" ") for doc in paras]
bm25 = BM25Okapi(tokenized_corpus)

In [10]:
def get_key(val):
    for key, value in queries.items():
        if val == value:
            return key
bm25_results = {}
for q in queries.values():
  tokenized_query = q.split(" ")
  doc_scores = bm25.get_scores(tokenized_query)
  l = sorted(range(len(doc_scores)), key=lambda i: doc_scores[i])[-5:]
  id = get_key(q)
  d = {}
  for v in l:
    d[str(v)] = doc_scores[v]
  bm25_results[id] = d

In [ ]:
model = DRES(models.SentenceBERT('sentence-transformers/multi-qa-mpnet-base-dot-v1'), batch_size=16)
retriever = EvaluateRetrieval(model, score_function="dot")

results = retriever.retrieve(corpus, queries, bm25_result = bm25_results, k_values = [5])

In [ ]:
top_k_accuracy(
        qrels, 
        results, 
        [1])